In [ ]:
from CA import *
from initial_state_generation import *
import matplotlib.pyplot as plt

if __name__ == "__main__":
    # Example usage
    np.random.seed(42)
    width, height, ground_height = 21, 101, 101*.1
    
    initial_state = generate_initial_slope(height, width, ground_height, noise_amplitude = 0.2, noise_type = 'white')
    
    output_file = '../videos/water_simulation.mp4'
    ca = CA(width, height, initial_state, neighbor_list=BOTTOM_NEIGHBORS)
    ca.run_simulation(1000, show_live=True)
    
    plt.imshow(ca.grid[:,:,GROUND_HEIGHT] - initial_state[:,:,GROUND_HEIGHT])
    plt.colorbar()
    
    plt.savefig('../data/test.png')
    
    

In [ ]:
from cpp_modules import fastCA

np.random.seed(42)
width, height, ground_height, num_steps = 21, 101, 101*.1, 10000

initial_state = generate_initial_slope(height, width, ground_height, noise_amplitude = 0.2, noise_type = 'white')

add_central_flow(initial_state, flow_amount=1)


grids = np.zeros([num_steps, height, width, NUM_CELL_FLOATS])
grids[0] = initial_state


# plt.imshow(grids[-1,:,:,GROUND_HEIGHT] - initial_state[:,:,GROUND_HEIGHT])
# plt.imshow(grids[0,:,:,WATER_HEIGHT] )
# plt.colorbar()


params = {
    "EROSION_K": EROSION_K,
    "EROSION_C": EROSION_C,
    "EROSION_n": N,
    "EROSION_m": EROSION_EXPONENT,
}

fastCA.simulate(grids, params)

# print(grids)
# save_video(grids, 'videos/cpp_test.mp4')
# print(grids)

# # plt.imshow(grids[0,:,:,WATER_HEIGHT] )
# plt.imshow(grids[-1,:,:,WATER_HEIGHT] - grids[0,:,:,WATER_HEIGHT])
plt.imshow(np.log10(grids[-1,:,:,WATER_HEIGHT] * (grids[-1,:,:,WATER_HEIGHT] > 1e-6)) )
# plt.imshow(grids[-1,:,:,GROUND_HEIGHT] - grids[0,:,:,GROUND_HEIGHT])
plt.colorbar()
plt.savefig('../data/cpptest.png')
plt.show()

for gen in range(10):
    grids[0] = grids[-1]
    add_central_flow(grids[0], flow_amount=1)

    fastCA.simulate(grids, params)

    # plt.imshow(grids[-1,:,:,WATER_HEIGHT] - grids[0,:,:,WATER_HEIGHT])
    plt.imshow(np.log10(grids[-1,:,:,WATER_HEIGHT] * (grids[-1,:,:,WATER_HEIGHT] > 1e-6)) )
    # plt.imshow(grids[-1,:,:,GROUND_HEIGHT] - grids[0,:,:,GROUND_HEIGHT])
    plt.colorbar()
    plt.show()

